In [ ]:
%cd /Users/georgi/dev/dialogue_modeling

In [ ]:
from supporting_classifiers.agreement_classifier import *
from solution_tracker.simple_sol import solution_tracker, process_raw_to_solution_tracker

In [ ]:
import spacy
import string
from read_data import read_solution_annotaions, read_wason_dump, read_3_lvl_annotation_file
import pandas as pd
from featurisers.raw_wason_featuriser import get_y

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
hierch_data = read_3_lvl_annotation_file('3lvl_anns.tsv')


for a in hierch_data:
    a.preprocess_everything(nlp)

raw_data = read_wason_dump('data/all/')

conversations_to_process = {}
for ann in hierch_data:
    raw = [r for r in raw_data if r.identifier == ann.identifier][0]
    ann.raw_db_conversation = raw.raw_db_conversation
    conversations_to_process[ann.identifier] = ann

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
from solution_tracker.augment_with_solution import merge_with_solution
import plotly.graph_objects as go
from featurisers.raw_wason_featuriser import calculate_stats, preprocess_conversation_dump

In [ ]:
def process_in_window(conversation, threshold, window_size, bad_cases=False): 
    output = []
    for index, message in enumerate(messages_supervised):
        begining = 0 if index-window_size < 0 else index-window_size
        running_window = messages_supervised[begining:index]
        if len(running_window) == 0:
            continue
        diff = running_window[-1].annotation['team_performance'] - running_window[0].annotation['team_performance']
        if bad_cases:
            if diff > threshold:
                continue
        else:
            if diff < threshold:
                continue
                
        inner_window = []
        for item in running_window:
            inner_window.append({'window_improvement': diff ,'message': item.content, 'annotation': item.annotation,
                          'improvement': message.annotation['performance_change']})
        output.append(inner_window)
      
    return output

In [ ]:
good_cases = []
bad_cases = []
for key, conv in conversations_to_process.items():
    messages_supervised = merge_with_solution(conv)
    output_good = process_in_window(messages_supervised, 0.1, 6)
    output_bad = process_in_window(messages_supervised, -0.1, 6, True)

    good_cases.extend(output_good)
    bad_cases.extend(output_bad)


In [ ]:
len(good_cases)

In [ ]:
len(bad_cases)

In [ ]:
orderred_good = sorted(good_cases, key=lambda k: k[0]['window_improvement'], reverse=True) 


In [ ]:
orderred_good

In [ ]:
def print_orderred(orderd):
    for item in orderd:
        for message in item:
            print("{}\t{}\t{}".format(message['message'], message['window_improvement'], message['annotation']))
        print('~~~~~~~~~~~~~\t~~~~~~~~~~\t~~~~~~~~~~~')

In [ ]:
print_orderred(orderred_good)

In [ ]:
print_orderred(good_probing)

In [ ]:
len(good_probing)

In [ ]:
orderred_good_probing = sorted(good_probing, key=lambda k: k['improvement'], reverse=True) 


In [ ]:
orderred_good_probing

In [ ]:
print_orderred(bad_cases)

In [ ]:
orderred_bad = sorted(bad_cases, key=lambda k: k[0]['improvement'], reverse=False) 


In [ ]:
print_orderred(orderred_bad)

In [ ]:
orderred_bad

In [ ]:
bad_probing = [g for g in bad_cases if g['annotation']['type'] == 'Probing']

In [ ]:
print_orderred(bad_probing)

In [ ]:
orderred_bad_probing = sorted(bad_probing, key=lambda k: k['improvement'], reverse=False) 
orderred_bad_probing